# Create Solr Core

In [123]:
# Delete any existing core
!curl -X GET "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=search_reddit&deleteInstanceDir=true&deleteDataDir=true"

{
  "responseHeader":{
    "status":0,
    "QTime":67
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    59  100    59    0     0    217      0 --:--:-- --:--:-- --:--:--   216
100    59  100    59    0     0    217      0 --:--:-- --:--:-- --:--:--   216


In [124]:
import shutil

# Create directory
shutil.copytree("C:\\solr-9.5.0-slim\\server\\solr\\configsets\\_default", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit")

'C:\\solr-9.5.0-slim\\server\\solr\\search_reddit'

In [125]:
# Create solr core with default settings
!curl -X GET "http://localhost:8983/solr/admin/cores?action=CREATE&name=search_reddit&instanceDir=search_reddit"

{
  "responseHeader":{
    "status":0,
    "QTime":160
  },
  "core":"search_reddit"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0    229      0 --:--:-- --:--:-- --:--:--   229


# Solr recommends we use HTTP requests to edit schema. Because of data integrity issues

In [126]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   574  100   574    0     0   2690      0 --:--:-- --:--:-- --:--:--  2694


# Add custom schema & index

https://solr.apache.org/guide/6_6/field-properties-by-use-case.html#field-properties-by-use-case
https://solr.apache.org/guide/6_6/field-type-definitions-and-properties.html#field-type-definitions-and-properties

## With only basic cleaning done beforehand, the rest have to be done in Solr. Therefore we define a custom field type for text

- https://solr.apache.org/guide/6_6/filter-descriptions.html

text_general for reference
```
</fieldType>
  <fieldType name="text_general" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter name="lowercase"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer name="standard"/>
      <filter words="stopwords.txt" name="stop" ignoreCase="true"/>
      <filter ignoreCase="true" synonyms="synonyms.txt" name="synonymGraph" expand="true"/>
      <filter name="lowercase"/>
    </analyzer>
</fieldType>
```

text_reddit custom field type
- Query and index filters do not need to be the same
- Synonyms are used to expand search results for query terms only
- NGram filter is used to create n-grams for index terms only
- The rest can be the same as text_general
```
  <fieldType name="text_reddit" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
      <filter class="solr.NGramFilterFactory" maxGramSize="20" minGramSize="2"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer class="solr.WhitespaceTokenizerFactory"/>
      <filter class="solr.StopFilterFactory" words="stopwords.txt" ignoreCase="true"/>
      <filter class="solr.SynonymGraphFilterFactory" ignoreCase="true" synonyms="synonyms.txt" expand="true"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
    </analyzer>
```

In [127]:
# Tried with General tokenizer but didnt work
# !curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

# Try with Whitespace tokenizer
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":2,"maxGramSize":20}]},"queryAnalyzer":{"tokenizer":{"class":"solr.WhitespaceTokenizerFactory"},"filters":[{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SynonymGraphFilterFactory","ignoreCase":true,"synonyms":"synonyms.txt","expand":true},{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.SnowballPorterFilterFactory","language":"English"}]}}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":138
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   782  100    60  100   722    171   2058 --:--:-- --:--:-- --:--:--  2227
100   782  100    60  100   722    171   2058 --:--:-- --:--:-- --:--:--  2227


## Field types explained
- indexed -> If true, the value of the field can be used in queries to retrieve matching documents.
- stored -> If true, the actual value of the field can be retrieved by queries.

Our search engine primarily searches by comment & post texts

```
<field name="_nest_path_" type="_nest_path_"/>
<field name="_root_" type="string" indexed="true" stored="false" docValues="false"/>
<field name="_text_" type="text_general" multiValued="true" indexed="true" stored="false"/>
<field name="_version_" type="plong" indexed="false" stored="false"/>
<field name="author" type="text_general"/>
<field name="created_utc" type="pdates"/>
<field name="edited" type="pdates"/>
<field name="id" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="num_comments" type="pdoubles"/>
<field name="permalink" type="text_general"/>
<field name="post_id" type="text_general"/>
<field name="score" type="plongs"/>
<field name="subreddit_name" type="text_general"/>
<field name="text" type="text_general"/>
<field name="type" type="text_general"/>
<field name="upvote_ratio" type="pdoubles"/>
<field name="url" type="text_general"/>
```

In [128]:
# Author, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"author","type":"string","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# Text, searched and retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"text","type":"text_reddit","stored":true,"indexed":true,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# created_utc, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"created_utc","type":"pdates","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# edited, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"edited","type":"pdates","stored":true,"indexed":true}}" http://localhost:8983/api/cores/search_reddit/schema

# id, retrieved, unique key
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"id","type":"string","stored":true,"indexed":true,"multiValued":false,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# num_comments, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"num_comments","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# permalink, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"permalink","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# score, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"score","type":"plong","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# subreddit_name, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"subreddit_name","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# upvote_ratio, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"upvote_ratio","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# url, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"url","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# type, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"type","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# post_id, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"post_id","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":124
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   172  100    60  100   112    178    333 --:--:-- --:--:-- --:--:--   511


{
  "responseHeader":{
    "status":0,
    "QTime":113
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   139  100    60  100    79    181    239 --:--:-- --:--:-- --:--:--   421


{
  "responseHeader":{
    "status":0,
    "QTime":108
  }
}


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   123  100    59  100    64    192    209 --:--:-- --:--:-- --:--:--   403


{
  "responseHeader":{
    "status":0,
    "QTime":90
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   123  100    59  100    64    191    207 --:--:-- --:--:-- --:--:--   398


{
  "responseHeader":{
    "status":0,
    "QTime":102
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   127  100    60  100    67    190    213 --:--:-- --:--:-- --:--:--   404


In [129]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"author",
    "type":"string",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"created_utc",
    "type":"pdates",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"edited",
    "type":"pdates",
    "indexed":true,
    "stored":true
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  },{
    "name":"num_comments"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2023  100  2023    0     0   9475      0 --:--:-- --:--:-- --:--:--  9497


# Ingest data

In [130]:
# add document to solr
!curl "http://localhost:8983/solr/search_reddit/update?commit=true" --data-binary @data/cleaned_combined_data.csv -H "Content-type:application/csv"

{
  "responseHeader":{
    "status":0,
    "QTime":7744
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 20 13.7M    0     0   20 2816k      0  11.9M  0:00:01 --:--:--  0:00:01 11.9M
 55 13.7M    0     0   55 7744k      0  3474k  0:00:04  0:00:02  0:00:02 3474k
 75 13.7M    0     0   75 10.3M      0  3087k  0:00:04  0:00:03  0:00:01 3087k
100 13.7M    0     0  100 13.7M      0  3113k  0:00:04  0:00:04 --:--:-- 3113k
100 13.7M    0     0  100 13.7M      0  2547k  0:00:05  0:00:05 --:--:-- 2546k
100 13.7M    0     0  100 13.7M      0  2153k  0:00:06  0:00:06 --:--:-- 1784k
100 13.7M    0     0  100 13.7M      0  1863k  0:00:07  0:00:07 --:--:-- 1187k
100 13.7M  100    61  100 13.7M      7  1767k  0:00:08  0:00:07  0:00:01  759k


# Test query data

- https://solr.apache.org/guide/6_6/the-standard-query-parser.html

## Select with keyword search

In [131]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats*&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":3,
    "params":{
      "q":"text:congrats*",
      "rows":"10"
    }
  },
  "response":{
    "numFound":561,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"pboksz",
      "text":["Congrats! That is one impressive reward! This is such a fun car and it will be great for hauling the two littles one!"],
      "created_utc":"2022-03-29T03:02:07Z",
      "id":"i2h052j",
      "permalink":"/r/BMWi3/comments/tqghan/my_reward_for_finally_finishing_my_bachelors/i2h052j/",
      "score":10,
      "subreddit_name":["BMWi3"],
      "type":["comment"],
      "post_id":["t3_tqghan"],
      "_version_":1794344969465495558
    },{
      "author":"jazzcat007",
      "text":["Youâ€™re gonna love it! Congrats!"],
      "created_utc":"2022-03-29T07:48:17Z",
      "id":"i2i2ebt",
      "permalink":"/r/BMWi3/comments/tqghan/my_reward_for_finally_finishing_my_bachelors/i2i2ebt/",
      "score":2,
      "subreddit_name":["BMWi3"],
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4519  100  4495  100    24  20340    108 --:--:-- --:--:-- --:--:-- 20540


## Search with timeframe range filter

In [132]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5243  100  5175  100    68  24484    321 --:--:-- --:--:-- --:--:-- 24848



  "responseHeader":{
    "status":0,
    "QTime":2,
    "params":{
      "q":"created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":1923,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"billbucket",
      "text":["Tesla's Summon feature was very useful today..."],
      "created_utc":"2018-01-25T07:13:35Z",
      "id":"7srdiw",
      "num_comments":1050.0,
      "permalink":"/r/teslamotors/comments/7srdiw/teslas_summon_feature_was_very_useful_today/",
      "score":49182,
      "subreddit_name":["teslamotors"],
      "upvote_ratio":0.9,
      "url":"https://gfycat.com/ReliableSecretJunebug",
      "type":["post"],
      "_version_":1794344969460252693
    },{
      "author":"TheMightyCraken",
      "text":["Elon Musk confirms secret Tesla â€˜Carless Driverâ€™ project"],
      "created_utc":"2018-01-30T05:20:51Z",
      "id":"7tvnx6",
      "num_comments":394.0,
      "permalink":"/r/teslamotors/co

## Combination of conditions

In [133]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":3,
    "params":{
      "q":"text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":4,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"the8king",
      "text":["Wow, congrats!! I wonder how much that was vs a gas roadtrip!"],
      "created_utc":"2017-03-28T23:00:50Z",
      "id":"dfijlrq",
      "permalink":"/r/teslamotors/comments/61yrt5/we_just_drove_from_san_francisco_to_new_york_city/dfijlrq/",
      "score":2,
      "subreddit_name":["teslamotors"],
      "type":["comment"],
      "post_id":["t3_61yrt5"],
      "_version_":1794344975046017033
    },{
      "author":"Randomd0g",
      "text":["You made it! Congrats, friends. About what percentage of the total drive would you say was done with AP?"],
      "created_utc":"2017-03-28T21:10:44Z",
      "id":"dfiegxk",
      "permalink":"/r/teslamotors/comments/61yrt5/we_just_drov

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2351  100  2264  100    87  10275    394 --:--:-- --:--:-- --:--:-- 10686


# Add spellcheck suggestions in response if query returns less results

- https://solr.apache.org/guide/8_1/spell-checking.html

DirectSolrSpellChecker
```
<searchComponent name="spellcheck" class="solr.SpellCheckComponent">
  <lst name="spellchecker">
    <str name="name">default</str>
    <str name="field">name</str>
    <str name="classname">solr.DirectSolrSpellChecker</str>
    <str name="distanceMeasure">internal</str>
    <float name="accuracy">0.5</float>
    <int name="maxEdits">2</int>
    <int name="minPrefix">1</int>
    <int name="maxInspections">5</int>
    <int name="minQueryLength">4</int>
    <float name="maxQueryFrequency">0.01</float>
    <float name="thresholdTokenFrequency">.01</float>
  </lst>
</searchComponent>
```

Add to request to use spellcheck
```
<str name="spellcheck">true</str>
```

Thus, our spellchecker will be

```
  <searchComponent name="spellcheck" class="solr.SpellCheckComponent">

    <str name="queryAnalyzerFieldType">text_reddit</str>

    <!-- Multiple "Spell Checkers" can be declared and used by this
         component
      -->

    <!-- a spellchecker built from a field of the main index -->
    <lst name="spellchecker">
      <str name="name">default</str>
      <str name="field">text</str>
      <str name="classname">solr.DirectSolrSpellChecker</str>
      <!-- the spellcheck distance measure used, the default is the internal levenshtein -->
      <str name="distanceMeasure">internal</str>
      <!-- minimum accuracy needed to be considered a valid spellcheck suggestion -->
      <float name="accuracy">0.5</float>
      <!-- the maximum #edits we consider when enumerating terms: can be 1 or 2 -->
      <int name="maxEdits">2</int>
      <!-- the minimum shared prefix when enumerating terms -->
      <int name="minPrefix">1</int>
      <!-- maximum number of inspections per result. -->
      <int name="maxInspections">5</int>
      <!-- minimum length of a query term to be considered for correction -->
      <int name="minQueryLength">4</int>
      <!-- maximum threshold of documents a query term can appear to be considered for correction -->
      <float name="maxQueryFrequency">0.01</float>
      <!-- uncomment this to require suggestions to occur in 1% of the documents
        <float name="thresholdTokenFrequency">.01</float>
      -->
    </lst>
```

# Restart core to apply changes
## Somehow http request for this dosent work, manually edit and refresh core

In [134]:
# # Edit spellchecker to work for text_reddit, the custom type we created
# !curl -X POST -H "Content-type: application/json" --data-binary "{"add-searchcomponent": {"name": "spellcheck2", "class": "solr.SpellCheckComponent", "config": {"queryAnalyzerFieldType": "text_reddit", "spellchecker": {"name": "default", "field": "text", "classname": "solr.DirectSolrSpellChecker", "distanceMeasure": "internal", "accuracy": 0.5, "maxEdits": 2, "minPrefix": 1, "maxInspections": 5, "minQueryLength": 4, "maxQueryFrequency": 0.01}}}}" http://localhost:8983/api/cores/search_reddit/config

## Refresh core

In [135]:
!curl -X POST "http://localhost:8983/solr/admin/cores?action=RELOAD&core=search_reddit&wt=json"

{
  "responseHeader":{
    "status":0,
    "QTime":481
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    60  100    60    0     0     85      0 --:--:-- --:--:-- --:--:--    85
100    60  100    60    0     0     85      0 --:--:-- --:--:-- --:--:--    85


## Test spellchecker

In [136]:
!curl "http://localhost:8983/solr/search_reddit/spell?indent=true&spellcheck.q=graet&spellcheck=true&spellcheck.collate=true"

{
  "responseHeader":{
    "status":0,
    "QTime":17
  },
  "response":{
    "numFound":0,
    "start":0,
    "numFoundExact":true,
    "docs":[ ]
  },
  "spellcheck":{
    "suggestions":["graet",{
      "numFound":10,
      "startOffset":0,
      "endOffset":5,
      "origFreq":0,
      "suggestion":[{
        "word":"great",
        "freq":1379
      },{
        "word":"grant",
        "freq":43
      },{
        "word":"grate",
        "freq":18
      },{
        "word":"garet",
        "freq":9
      },{
        "word":"greet",
        "freq":4
      },{
        "word":"grat",
        "freq":1200
      },{
        "word":"gret",
        "freq":75
      },{
        "word":"grade",
        "freq":117
      },{
        "word":"giant",
        "freq":75
      },{
        "word":"g&aut",
        "freq":59
      }]
    }],
    "correctlySpelled":false,
    "collations":[ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   890  100   890    0     0   3744      0 --:--:-- --:--:-- --:--:--  3739
